# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

This is a classification problem. The purpose of the project is to determine which students will require intervention. Hence, the desired output groups will be students who need intervention and students who do not require intervention, 2 specific labels. Also, the target column is made up of categorical data instead of continuous numerical data. 

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from IPython.display import Image
from IPython.core.display import HTML 


# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [4]:

# TODO: Calculate number of students
# Total number of students can be calculated by returning the total number of rows 
n_students = student_data.shape[0]

# TODO: Calculate number of features
# Total number of features can be calculated by returning the total number of columns with the passed column removed
n_features = student_data.shape[1]-1

# In the following calculations, we first identify each of the sub groups within the 'passed' column before counting them 
# TODO: Calculate passing students
n_passed = student_data.passed[student_data.passed == 'yes'].count()

# TODO: Calculate failing students
n_failed = n_failed = student_data.passed[student_data.passed == 'no'].count()

# TODO: Calculate graduation rate
# Calculate graduation rate by standard multiplication function here instead of numpy is more elegant
grad_rate = 100.*n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [5]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [6]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [7]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import StratifiedShuffleSplit as SSS

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = 95

# TODO: Shuffle and split the dataset into the number of training and testing points above

def stratify_and_shuffle(X,y,num_train):
    sss = SSS(y, 1, train_size=num_train, test_size=num_test)

    # only one iteration
    for train_index, test_index in sss:
        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        
    return X_train, X_test, y_train, y_test
    
X_train, X_test, y_train, y_test = stratify_and_shuffle(X_all, y_all, num_train)
# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

The supervised learning models that are appropriate are:
1) Decision Tree Classifier
2) Gaussian Naive Bayes
3) Support Vector Classifier

General Applications:
1) Decision Tree Classifier : Operations Research, especially in the decision analysis process i. e To identify a strategy that most likely reaches a desired goal
2) Naive Bayes: Text categorisation e.g document classification, spam filtering. It is also used in automatic medical diagnosis.
3) Support Vector Classifier: Data classification and many industrial scale applications. Classification, Regression and outliers detection 

Strength and Weaknesses:

DECISION TREE CLASSIFIER

Strengths:

1) Easy to understand 
2) Sturdy model and require little preparation( not requires data normalisation, dummy variables, outlier, missing values resolution and sustains model generation assumption violations )
3) It can scale logarithmically with data size
4) Especially versatile when used on dataset with categorical and numerical values

Weaknesses:

1) The use of Decision Trees may lead to overfitting.
2) Decision Trees need to be rebuilt when there is new data added/removed into the database. Hence it is not suitable for analysis that involves live continuous update of the dataset. 
3) Decision tree learners will create biased trees if some classes dominate.(recommended to balance the dataset prior to fitting with the decision tree.)


NAIVE BAYES:

Strengths

1) It can estimate parameters even with small data size effectively
2) Highly scalable and generates models extremely fast

Weakness
1) Predicted probability maybe inaccurately estimated and should not be always taken seriously


SUPPORT VECTOR CLASSIFIERS:

Strengths

1) It is effective in high dimensional space (it could still provide good prediction when the number of features is greater than the number of samples) and it is also capable of modelling complex separating hyperplanes
2) It is memory efficient, as only uses a subset of training point to generate the hyperplanes.
3) We can use different kernel function to model different separating hyperplanes

Weakness

1) It requires a lot of memory and cpu time 
2) It does not directly provide probability estimates. 
3) It is expensive when used for 5-fold cross validation and multi class problem
4) If number of dimensions is much greater than training samples, its performance maybe compromised




What the initial statistics tell us about our dataset:

1) There is a greater proportion of students passing than failing, hence dataset is slightly skewed.
2) There is a good proportion of features to the sample data. (more sample data available compared to number of features)
3) Data set is small.


Reasons for choice of Decision Tree Classifier:
a) It is a classifier that can easily be visualized, hence it will be easy to understand and interprete.
b) It is capable of binary classification, this is useful since the result of our prediction is binary(whether a student passes or fails). 
c) It requires little data preparation.
    
Reasons for choice of Naive Bayes:

1) It has a light memory and cpu footprint
2) It requires a small amount of training data to estimate the necessary parameters.
    
Reasons for choice of Support Vector Classifier:

1) SVCs work well when the size of the dataset is small
  
   



### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [9]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = DTC(random_state=42)
clf_B = GNB(random_state=42)
clf_C = SVC(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size

# Noted that I may use a loop to go through the algorithms but decided 
# to leave them to clearly show 9 different results as specified by the rubric.

#for clf in [clf_A, clf_B, clf_C]:
    #print "\n{}: \n".format(clf.__class__.__name__)
    #for n in [100, 200, 300]:
        #train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


#Train_predict for DTC
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

#Train_predict for GNB
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

#Train_predict for SVC
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)


Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0015 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7667.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0052 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7258.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0036 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7727.
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8033.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.7368.
Training a GaussianNB using a training set s

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer A - Decision Tree Classifier**  

| Training Set Size | Training Time   | Prediction Time (test) | F1 Score (train)| F1 Score (test) |
| :---------------: | :-------------: | :--------------------: | :-------------: | :-------------: |
| 100               | 0.0045 seconds  | 0.0004 seconds         | 1.0000         |0.6833           |
| 200               | 0.0033 seconds  | 0.0005 seconds         | 1.0000         |0.7460           |
| 300               | 0.0044 second   | 0.0004 seconds         | 1.0000         |0.7231           |

** Classifer B - Gaussian Naive Bayes Classifier **  

| Training Set Size | Training Time   | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :-------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0019 seconds  | 0.0008 seconds         | 0.8475           |  0.6726         |
| 200               | 0.0020 seconds  | 0.0007 seconds         | 0.8261           |  0.7727         |
| 300               | 0.0023 seconds  | 0.0007 seconds         | 0.7913           |  0.7597         |

** Classifer C - Support Vector Classifier**  

| Training Set Size | Training Time   | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :-------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0027 seconds  | 0.0016 seconds         | 0.8936           | 0.7973          |
| 200               | 0.0065 seconds  | 0.0024 seconds         | 0.8904           | 0.8000          |
| 300               | 0.0127 seconds  | 0.0034 seconds         | 0.8671           | 0.7815          |       

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

From the experiments conducted earlier, the model most appropriate based on the available data is support vector classifier. The experiment results above is very much within expectations of what was described in question 2 above.
Before we explain the results, let us briefly understand how to interprete the evaluation metrics. The different training set sizes are factored into the model such that we can make comparisons of the training and prediction time taken as well as the consistency of the F1 scores as training sizes increases. Generally we look out for consistent F1 scores even as training sizes increases.

Training and Prediction time estimates the duration that is required for these tests to be carried out. We look out for the shortest time possible for these tests to be carried out but expects duration to increase as training set size increases. The faster the time taken for the training and the test, the less costly it will be in terms of resource expanded and opportunity cost incurred in the event the prediction time affects the costs of operation (e.g electronic trading of financial derivatives)

The F1 score is a measure of accuracy of these tests with 0 being no accuracy at all an 1 being extremely accurate and precise. F1 scores between training data is compared with the test data to ensure that when testing the model on data that was not used to train the model (new and unexpected data), the F1 score remains close to 1- meaning, the trained model can actually predict future values that are accurate.  

The Decision Tree Classifier returned a perfect training F1 score in all three training set sizes but returned the poorest set of testing F1 score with a range of 0.683 to 0.746. This can be explained by overfitting. As the classifier itself is prone to the problem of overfitting in presence of a sufficiently large number of features, we can thus conclude that the number of features in this data set is more than the ideal number for this particular size of data results in overfitting. F1 scores for this classifier is also the most inconsistent. It is also worth noting that this model has the fastest prediction time amongst the three. Overfitting can be solved but will not be explored in this experiment.

The Naive Bayes Classifier returned the fastest training time ranging from 0.0019 to 0.0023 seconds. This is inline with expectations as described in question 2 that this classifier will be able to generate models extremely fast.This Classifier is faster than SVC by almost 2 to 3 times. The tight range of training time also explains its high scalability. Unfortunately, this model returns the poorest range of F1 test scores between 0.6726 and 0.777.

The Support Vector Classifier has the longest training and prediction time but gives the best F1 scores that is both accurate (high in value) and consistent within the 3 training sets and when comparing the scores between test and training set. 

In conclusion, SVC is the most ideal and appropriate model. In this particular project, we are to accurately determine which students are likely to fail to graduate and intervene beforehand, thus accuracy is of the highest priority for this purpose. Besides, for all 3 models, the time take for both prediction and training are less than 1 second. The evaluation is conducted only after the data set has been collected for some time and is not ongoing. Thus, with a processing time of less than 1 second, I would say even the slowest SVC model is pretty fast for this purpose.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

A SVC is a type of linear separator. Suppose we want to split students predicted to fail from students who are likely to pass by drawing a line. Notice that there are an infinite number of lines that will accomplish this task. SVCs, in particular, find the "maximum-margin" line - the middle line. We draw a straight line through the data(all the students mixed up) through the middle to separate it into two classes, pass and fail and in the process try to maximize the diagonal distance between the 2 groups.To draw the maximum margin line, only the data points closest to the line matter. These points are known as "support vectors" which gives SVC its name as seen in the image below.

Image(url= " https://udacity-github-sync-content.s3.amazonaws.com/_imgs/372/1468851732/SVM_2.png")



Thus SVC will first generate a straightline as the first set of information passes through. As it undergoes training with more new data passed along the classifier, it will attempt to optimize the maximum margin line and it may change shape, into curves in the process as various points close to the support vectors are "lifted" to a higher dimension onto a hyperplane (no longer 2D). Through this repetitive process, the maximum margin line is optimized and will be capable of separating the 2 groups very accurately and clearly as the margin width is being maximally stretched(see picture above) within its optimized parameters. Thus this will result in a distinct separator that is very capable of accurately separating students who are likely to graduate and those who will not. 

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [11]:
# Will Revisit the script to apply feature scaling at a later time
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV as GSCV
from sklearn.metrics import make_scorer as ms

# TODO: Create the parameters list you wish to tune
parameters = [{'C':[1,10,50,100,200,300,400,500,1000],
               'gamma':[1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1],
               'kernel': ['rbf']}]


# TODO: Initialize the classifier
clf_SVC = SVC()
            
# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = ms(f1_score, pos_label = "yes")


# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GSCV(estimator = clf_SVC, param_grid = parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf_SVC = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf_SVC, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf_SVC, X_test, y_test))

Made predictions in 0.0088 seconds.
Tuned model has a training F1 score of 0.9781.
Made predictions in 0.0029 seconds.
Tuned model has a testing F1 score of 0.8153.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

Tuned model has a training F1 score of 0.9781 and a testing F1 score of 0.8153. Both the tuned training and testing F1 scores are higher than the untuned model, comparing tuned to untuned training scores of 0.9781 to the highest 0.8936 and tuned to untuned test scores of 0.8153 to 0.800. Thus the tuned model will easily generate F1 scores that are higher than the average of those in an untuned model. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.